Vodafone PR lauch V1.0

In [ ]:
#liberary
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import math
import xlwings as xw
import datetime
import glob
import os


In [ ]:
def log_in(Web,person):
    bookmark = {'ibuy': "http://w3.huawei.com/ibuy/menu.do?actionFlag=getMenu&isIntroStr=yes", 'ERP_PM': "http://w3.huawei.com/pm/", 'ERP_PC': "http://w3.huawei.com/pc/#!common/pages/home.html",'ISDP': "http://app.huawei.com/sdcp/portalNew",'ECOR': "http://w3.huawei.com/ewe/ecor/#!ecor/ukvr/approve/myToDo.html"}
    ID_password={}
    try:
        url=bookmark[Web]
    except KeyError :
        print("please fill in correct web:'ibuy','ERP_PM','ERP_PC','ISDP','ECOR'")
    try:
        ID=ID_password[person][0]
        password=ID_password[person][1]
    except KeyError :
        print("please fill in correct name:'Yifan','Yuqi','Wenji','Junjun','Mengmeng','Yizhe'")    
    driver.set_page_load_timeout(10)
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)
    driver.find_element_by_name("uid").send_keys(ID)
    driver.find_element_by_name("password").send_keys(password)
    driver.find_element_by_name("password").send_keys(Keys.ENTER)

In [ ]:
def turn_list(x):
    y = list(set(list(x['Product Code'])))
    y.sort(reverse=False)
    return y
def turn_list_AP_code(x):
    y = list(set(list(x['Activity Code*'])))
    return y
def turn_list_AP_name(x):
    y = list(set(list(x['Activity Name*'])))
    return y
def HWPO_Check(x):
    if len(x)!= 14:
        y='000'+str(int(x))
    else:
        y=x
    return y

1.Read File & Check Trigger Status

In [ ]:
#Mutiple input
path_PR =r'D:\03.Purchase Reqeust\01.Request'
allFiles = glob.glob(path_PR+ "/*.xlsx")
Pr_template = pd.DataFrame()
list_files = []
for route in allFiles:
    Pr_temp = pd.read_excel(route, sheet_name='PR ENG',skiprows=[1,2],dtype=str)
    Pr_temp = Pr_temp.replace('nan', np.nan).dropna(how='all')
    Pr_temp['CONTRACT_NO']=Pr_temp['CONTRACT_NO'].map(lambda x : HWPO_Check(x))
    Pr_template=Pr_template.append(Pr_temp, ignore_index=True)
len(Pr_template)

In [ ]:
HW_PO_list=list(set(list(Pr_template['CONTRACT_NO'])))
len(HW_PO_list)

Check_trigger_rate_and_product service_code 

In [ ]:
#input: contract number, result: trigger rate,Service porduct code(ERP-PC)
PO_table_columns = ['HW_PO','Service_code', 'Trigger rate','Remark']
PO_trigger_table = pd.DataFrame(columns=PO_table_columns)
driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
log_in('ERP_PC',"Junjun")
time.sleep(3)
driver.find_element_by_css_selector("#leftSiteMapTree_12_span").click()
driver.find_element_by_css_selector("#leftSiteMapTree_17_span").click()
time.sleep(2)
count=0
One_shoot=True
for PO in HW_PO_list:
    driver.find_element_by_css_selector("#contractNo").send_keys(PO)
    driver.find_element_by_css_selector("#btnSearchSer").click()
    time.sleep(1)
    while One_shoot:
        try: 
            driver.find_element_by_css_selector("#srvMonitorGridIdPager > div > div.jalor-pager-group.pagesizes > select").click()
            driver.find_element_by_css_selector("#srvMonitorGridIdPager > div > div.jalor-pager-group.pagesizes > select").send_keys(Keys.DOWN)
            driver.find_element_by_css_selector("#srvMonitorGridIdPager > div > div.jalor-pager-group.pagesizes > select").send_keys(Keys.ENTER)
            One_shoot=False
            time.sleep(2)
        except Exception:
            break
    try:
        row=0
        while True:
            try:
                css_row='[_row='+'"'+str(row)+'"]'
                driver.find_element_by_css_selector(css_row).find_element_by_css_selector('[_col="1"]').click()
                Trigger_rate=driver.find_element_by_css_selector(css_row).find_element_by_css_selector('[_col="9"]')
                Product_code=driver.find_element_by_css_selector(css_row).find_element_by_css_selector('[_col="4"]')
                code=Product_code.text
                rate=Trigger_rate.text
                ERP_PM_data = pd.DataFrame([[PO,code,rate,""]], columns=PO_table_columns)
                PO_trigger_table=PO_trigger_table.append(ERP_PM_data)
                row=row+1
            except Exception:
                break
        driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="1"]').click()
        driver.find_element_by_css_selector("#serMonitorFormId > div.jalor-operate > span.init.jalor-button.reset").click() 
        time.sleep(1)
    except Exception:
        mark="N/A"
        ERP_PM_data = pd.DataFrame([[PO,"","",mark]], columns=PO_table_columns)
        PO_trigger_table=PO_trigger_table.append(ERP_PM_data)
        driver.find_element_by_css_selector("#serMonitorFormId > div.jalor-operate > span.init.jalor-button.reset").click()
        time.sleep(1)
    count=count+1
    if count % 10 == 0:
        print("Finished",count,"contracts, total:",len(HW_PO_list),"contracts")

print("Finish all",count,"contracts")
driver.quit()


In [ ]:
#Create all triggered PO table
def trigger_del(x):
    if x == '100.00':
        return True
    else : 
        return False 
PO_trigger_table['Delete']=PO_trigger_table['Trigger rate'].apply(trigger_del)
PO_trigger_table.set_index(keys=['HW_PO','Service_code'],inplace=True)
Search_PO_list=list(set(list(PO_trigger_table.index.get_level_values('HW_PO'))))
PO_invalid=pd.DataFrame(columns=['PO','Status'])
for PO in Search_PO_list:
    if False in list(PO_trigger_table.loc[PO]['Delete']):
        None
    else: 
        PO_data = pd.DataFrame([[PO,"ALL Triggered"]], columns=['PO','Status'])
        PO_invalid=PO_invalid.append(PO_data)
if len(PO_invalid)>0:
    print("Some PO can't be used, return to project team")
else: 
    print("Trigger rate test pass")

In [ ]:
#Create product code can't be used table
Product_used=pd.DataFrame(columns=['PO','Product_code','Status'])
for PO in Search_PO_list:
    Porduct_code_list=list(set(list(PO_trigger_table.loc[PO].index)))
    for Product_code in Porduct_code_list:
        trigger_list=list(set(list(pd.Series(PO_trigger_table.loc[PO].loc[Product_code]['Delete']))))
        if False in trigger_list:
            None
        else: 
            Product_code_data = pd.DataFrame([[PO,Product_code,"ALL Triggered"]], columns=['PO','Product_code','Status'])
            Product_used=Product_used.append(Product_code_data)

In [ ]:
#Write in EXCEL
now=datetime.datetime.now()
Trigger_file_name="Trigger_rate_check"+now.strftime("%H%M")+"_"+str(datetime.date.today())+'.xlsx'
Trigger_route='D:\\03.Purchase Reqeust\\02.Process\\'+Trigger_file_name
writer = pd.ExcelWriter(Trigger_route)
PO_trigger_table.to_excel(writer,'ERP_PC')
PO_invalid.to_excel(writer,'Used_PO')
Product_used.to_excel(writer,'Partial_Used')
writer.save()
PO_trigger_table

In [ ]:
#transfer PO list to str for input
def PO_list_to_str(x):
    all_PO_str=""
    for PO_n in range(0,len(x)):
        if PO_n == 0:
            all_PO_str=x[PO_n]
        else:
            all_PO_str=all_PO_str+","+x[PO_n]
    return all_PO_str 

In [ ]:
def Service_code_search(HWPO_str):
    #input: contract number, result: Service porduct code export file route(unpublished)(From ERP-PM)
    log_in('ERP_PM',"Yifan")
    menu=driver.find_element_by_css_selector('[data-menu-id="1001834"]')
    ActionChains(driver).move_to_element(menu).perform()
    hidden_submenu=driver.find_element_by_css_selector('[data-menu-id="1001839"]')
    ActionChains(driver).move_to_element(menu).click(hidden_submenu).perform()
    time.sleep(3)
    driver.find_element_by_css_selector('[id="hwContractNo"]').send_keys(HWPO_str)
    driver.find_element_by_css_selector('[id="showmodeldetail"]').click()
    time.sleep(1)
    driver.find_element_by_css_selector('[id="btnSearchModelDetail"]').click()
    driver.find_element_by_css_selector('[id="btnModelExport"]').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#jalor_dialog1 > div.jalor-dialog-body > div > div.jalor-dialog-content > a').click()
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    status=driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="5"]')
    while status.text!="Succeed":
        driver.find_element_by_css_selector("#_item1 > span").click()
        time.sleep(5)
        status=driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="5"]')
    Service_code_file_name_element=driver.find_element_by_css_selector(" #taskExportGridTable > tbody.igrid-data > tr:nth-child(2) > td.grid-cell.taskExportGrid-col2.left > a")
    Service_code_name= Service_code_file_name_element.text
    driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="2"]').find_element_by_css_selector('[class="link-special"]').click()
    time.sleep(5)
    driver.quit()
    route_Service_code_file="C:\\Users\\ywx610506\\Downloads\\" + Service_code_name
    return route_Service_code_file

driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
HWPO_input=PO_list_to_str(HW_PO_list)
route_HW_all=Service_code_search(HWPO_input)    
Service_Code = pd.read_excel(route_HW_all, sheet_name='Detail',dtype=str)

if len(Service_Code)>0:
    Service_Code['Huawei Contract No']=Service_Code['Huawei Contract No'].map(lambda x : HWPO_Check(x))
    HWPO_code_dict=dict(Service_Code.groupby('Huawei Contract No').apply(turn_list))
else:
    HWPO_code_dict={}
HWPO_code_dict

In [ ]:
Product_used.set_index(keys=['PO'],inplace=True)
if len(Product_used) > 0:
    Modify_PO_list=list(set(list(Product_used.index)))
    for PO in Modify_PO_list:
        Product_code_list=HWPO_code_dict[PO]
        Remove_list=list(set(list(pd.Series(Product_used.loc[PO]['Product_code']))))
        for code in Remove_list:
            Product_code_list.remove(code)
            HWPO_code_dict[PO]=Product_code_list
HWPO_code_dict

2. AP mapping-RFQ

In [ ]:
#4016737 MORAN
AP_route = 'D:\\03.Purchase Reqeust\\Database_and_Configuration\\AP_4016737.xls'

In [ ]:
#488690G Special
AP_route = 'D:\\03.Purchase Reqeust\\Database_and_Configuration\\AP_488690G.xls'

In [ ]:
AP_template = pd.read_excel(AP_route, sheet_name='Template',skiprows=[0],dtype=str)
AP_template = AP_template.replace('nan', np.nan).dropna(how='all')
item_Acode_mapping=dict(AP_template.groupby('Item').apply(turn_list_AP_code))
Acode_Aname_mapping=dict(AP_template.groupby('Activity Code*').apply(turn_list_AP_name))

Filling in Hang A template-RFQ

In [ ]:
#Fill in Hang A message, file:D:\03.Purchase Reqeust\Database_and_Configuration\ERP_PM_Hang_A_template_05102018.xlsm
def fill_in_HangA(dataframe, category):
    app=xw.App(visible=True,add_book=False)
    wb=app.books.open(r'D:\03.Purchase Reqeust\Database_and_Configuration\ERP_PM_Hang_A_template_05102018.xlsm')
    for x in range(0,len(dataframe)):
        sub_code="B"+str(int(x+4))
        DU_ID="C"+str(int(x+4))
        Task_code="E"+str(int(x+4))
        Task_type="F"+str(int(x+4))
        Task_name="G"+str(int(x+4))
        Qty="H"+str(int(x+4))
        service_code="I"+str(int(x+4))
        Strategy="J"+str(int(x+4))
        HWPO="O"+str(int(x+4)) 
        
        wb.sheets['Task'].range(sub_code).value=dataframe['SUB_PROJECT_CODE'][x]
        wb.sheets['Task'].range(DU_ID).value=dataframe['DU_CODE'][x]
        wb.sheets['Task'].range(Task_code).value=item_Acode_mapping[dataframe['ITEM_CODE'][x]][0]
        wb.sheets['Task'].range(Task_type).value="Activity"
        wb.sheets['Task'].range(Task_name).value=Acode_Aname_mapping[item_Acode_mapping[dataframe['ITEM_CODE'][x]][0]]
        wb.sheets['Task'].range(Qty).value=dataframe['QTY'][x]
        wb.sheets['Task'].range(service_code).value=dataframe['Service Product Code'][x]
        wb.sheets['Task'].range(Strategy).value="Subcontractor"
        wb.sheets['Task'].range(HWPO).value="'"+str(dataframe['CONTRACT_NO'][x])
    category_name=category
    now=datetime.datetime.now()
    HangA_file_name="HangA"+now.strftime("%H%M")+"_"+category_name+"_"+str(datetime.date.today())+'.xlsm'
    HangA_route='D:\\03.Purchase Reqeust\\02.Process\\'+HangA_file_name
    wb.save(HangA_route)
    wb.close()
    app.quit()
    return HangA_route
DF_HangA_ALL = Pr_template[['SUB_PROJECT_CODE', 'DU_CODE', 'ITEM_CODE','QTY','CONTRACT_NO']].copy()
DF_HangA_ALL['Service Product Code']=DF_HangA_ALL['CONTRACT_NO'].map(lambda x :str(HWPO_code_dict[x][0]))
Row_series=DF_HangA_ALL['Service Product Code']
DF_HangA_CT = DF_HangA_ALL[(Row_series=='9221250') | (Row_series=='9221251') | (Row_series=='9221519')].reset_index(drop=True)
DF_HangA_NonCT = DF_HangA_ALL[(Row_series!='9221250') & (Row_series!='9221251') & (Row_series!='9221519')].reset_index(drop=True)
if len(DF_HangA_CT)>0:
    HangA_CT_route=fill_in_HangA(DF_HangA_CT, 'CT')
if len(DF_HangA_NonCT)>0:
    HangA_NonCT_route=fill_in_HangA(DF_HangA_NonCT, 'NonCT')

Create A

In [ ]:
def fill_in_CreateA(route, category):
    HangA_temp = pd.read_excel(route, sheet_name='Task',skiprows=[0,1])
    HangA_temp = HangA_temp.dropna(how='all')
    Group_HangA = HangA_temp[['Sub-Project Code', 'Task Name', 'Task Code','Service Product Code']].copy()
    Group_HangA['Qty'] = int(20)
    Group_HangA['Total'] = Group_HangA.groupby(['Task Code', 'Service Product Code'])['Qty'].transform('sum')
    Group_HangA = Group_HangA.drop_duplicates(subset=['Task Code', 'Service Product Code']).reset_index(drop=True)
    app=xw.App(visible=True,add_book=False)
    wb=app.books.open(r'D:\03.Purchase Reqeust\Database_and_Configuration\ERP_PM_Create_A_template_08102018.xlsm')
    for x in range(0,len(Group_HangA)):
        sub_code="B"+str(int(x+4))
        Task_name="D"+str(int(x+4))
        Task_code="E"+str(int(x+4))
        service_code="F"+str(int(x+4))
        Strategy="H"+str(int(x+4))
        Qty="J"+str(int(x+4))
        
        wb.sheets['Sheet1'].range(sub_code).value=Group_HangA['Sub-Project Code'][x]
        wb.sheets['Sheet1'].range(Task_name).value=Group_HangA['Task Name'][x]
        wb.sheets['Sheet1'].range(Task_code).value=Group_HangA['Task Code'][x]
        wb.sheets['Sheet1'].range(service_code).value=Group_HangA['Service Product Code'][x]
        wb.sheets['Sheet1'].range(Strategy).value="Subcontractor"
        wb.sheets['Sheet1'].range(Qty).value=Group_HangA['Total'][x]
    category_name=category
    now=datetime.datetime.now()
    CreateA_file_name="CreatA"+now.strftime("%H%M")+"_"+category_name+"_"+str(datetime.date.today())+'.xlsm'
    CreateA_route='D:\\03.Purchase Reqeust\\02.Process\\'+CreateA_file_name
    wb.save(CreateA_route)
    wb.close()
    app.quit()
    return CreateA_route
if len(DF_HangA_CT)>0:
    CreateA_CT_route=fill_in_CreateA(HangA_CT_route, 'CT')
if len(DF_HangA_NonCT)>0:
    CreateA_NonCT_route=fill_in_CreateA(HangA_NonCT_route, 'NonCT')

Correct PR-template

In [ ]:
#Pr-template, correct Contract No. ,Department NO. ,Service Product Code
SN=0
for route in allFiles:
    Pr_temp = pd.read_excel(route, sheet_name='PR ENG',skiprows=[1,2],dtype=str)
    Pr_temp = Pr_temp.replace('nan', np.nan).dropna(how='all')
    Pr_temp['CONTRACT_NO']=Pr_temp['CONTRACT_NO'].map(lambda x : HWPO_Check(x))
    Pr_temp['DEPARTMENT_CODE']=Pr_temp['DEPARTMENT_CODE'].map(lambda x : "'0"+str(int(x)))
    app=xw.App(visible=True,add_book=False)
    wb=app.books.open(route)
    for x in range(0,len(Pr_temp)):
        DU_ID="O"+str(int(x+4))
        HWPO="E"+str(int(x+4))
        Departmet_code="N"+str(int(x+4))
        service_code="Z"+str(int(x+4))
        wb.sheets['PR ENG'].range(HWPO).value="'"+str(Pr_temp['CONTRACT_NO'][x])
        wb.sheets['PR ENG'].range(DU_ID).value=str(Pr_temp['DU_CODE'][x])
        wb.sheets['PR ENG'].range(Departmet_code).value=Pr_temp['DEPARTMENT_CODE'][x]
        wb.sheets['PR ENG'].range(service_code).value=HWPO_code_dict[Pr_temp['CONTRACT_NO'][x]][0]
    sub_con_name=Pr_temp['ENGINEERING_NAME'][0]
    now=datetime.datetime.now()
    PR_file_name="PR"+now.strftime("%H%M")+str(SN)+"_"+sub_con_name+"_"+str(datetime.date.today())+'.xlsx'
    Whole_route='D:\\03.Purchase Reqeust\\02.Process\\'+PR_file_name
    SN=SN+1
    wb.save(Whole_route)
    wb.close()
    app.quit()
    

Scenario Status

In [ ]:
driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
log_in('ERP_PM',"Yifan")
driver.implicitly_wait(10)
driver.find_element_by_css_selector("#searchText > ul > li.current > p").click()
driver.find_element_by_css_selector("#searchKey").send_keys('4016737')
time.sleep(1)
driver.find_element_by_css_selector("#searchKey").send_keys(Keys.ENTER)
driver.find_element_by_css_selector("#pm_subprojectTable > tbody.igrid-data > tr > td.grid-cell.pm_subproject-fzcol2.center > a").click()
time.sleep(2)
Scenario_place=driver.find_element_by_css_selector("#subProjectScenarioLi > span > label")
Scenario_name=Scenario_place.text
driver.quit()
Scenario_name

HangA_upload

In [ ]:
def upload_HangA(x):
    driver.implicitly_wait(10)
    log_in('ERP_PM',"Yifan")
    time.sleep(2)
    menu=driver.find_element_by_css_selector('[data-menu-id="1003331"]')
    ActionChains(driver).move_to_element(menu).perform()
    hidden_submenu=driver.find_element_by_css_selector('[data-menu-id="1003381"]')
    ActionChains(driver).move_to_element(menu).click(hidden_submenu).perform()
    driver.find_element_by_css_selector("#subProjectNo").send_keys('4016737')
    driver.find_element_by_css_selector('[id="btnSearch"]').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#searchSubProjectForWbsGrid_freeze_body > tbody > tr > td.gt-col-searchsubprojectforwbsgrid-chk > div > input').click()
    driver.find_element_by_css_selector('[id="btnNext"]').click()
    driver.switch_to.window(driver.window_handles[1])
    while True:
        try:
            WBS=driver.find_element_by_css_selector('[id="isc_0"]').find_element_by_css_selector('[id="isc_60"]').find_element_by_css_selector('[id="isc_5Ztable"]').find_element_by_css_selector('[id="PlanManagerView$Hirc_valueCell0"]')
            if WBS.text=="4016737":
                time.sleep(1)
                break
        except:
            time.sleep(3)
    driver.find_element_by_css_selector('#isc_2Q > table > tbody > tr > td > table').click()
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[2])
    file_input = driver.find_element_by_css_selector('[type="file"]')
    file_input.send_keys(x)
    driver.find_element_by_css_selector('[class="plupload_button plupload_start"]').click()
    driver.switch_to.window(driver.window_handles[0])
    menu=driver.find_element_by_css_selector('[data-menu-id="1320"]')
    ActionChains(driver).move_to_element(menu).perform()
    hidden_submenu=driver.find_element_by_css_selector('[data-menu-id="110291"]')
    ActionChains(driver).move_to_element(menu).click(hidden_submenu).perform()
    time.sleep(3)
    while True:
        driver.find_element_by_css_selector('[class="jalor-button default"]').click()
        time.sleep(1)
        file_name=driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="2"]')
        status=driver.find_element_by_css_selector('[_row="0"]').find_element_by_css_selector('[_col="4"]')
        if file_name.text==os.path.basename(x) and status.text=='Succeed':
            print("Hang_A_succeed")
            driver.quit()
            break            
        elif status.text!='In Process' and status.text!='Waiting':
            print("please check error massage")
            break
        else: 
            time.sleep(3)

if Scenario_name=='Energy,IBS,LS,Network,POD (Pure Equipment),Wireless (Including TK)' and len(DF_HangA_NonCT)>0:
    driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
    upload_HangA(HangA_NonCT_route)
elif Scenario_name=='Managed Operation (CT)' and len(DF_HangA_CT)>0:
    driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
    upload_HangA(HangA_CT_route)

Batch

In [ ]:
driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
driver.implicitly_wait(10)
log_in('ERP_PM',"Yifan")
time.sleep(2)
menu=driver.find_element_by_css_selector('[data-menu-id="1003331"]')
ActionChains(driver).move_to_element(menu).perform()
hidden_submenu=driver.find_element_by_css_selector('[data-menu-id="1003381"]')
ActionChains(driver).move_to_element(menu).click(hidden_submenu).perform()
driver.find_element_by_css_selector("#subProjectNo").send_keys('4016737')
driver.find_element_by_css_selector('[id="btnSearch"]').click()
time.sleep(1)
driver.find_element_by_css_selector('#searchSubProjectForWbsGrid_freeze_body > tbody > tr > td.gt-col-searchsubprojectforwbsgrid-chk > div > input').click()
driver.find_element_by_css_selector('[id="btnNext"]').click()
driver.switch_to.window(driver.window_handles[1])    
while True:
    try:
        WBS=driver.find_element_by_css_selector('[id="isc_0"]').find_element_by_css_selector('[id="isc_60"]').find_element_by_css_selector('[id="isc_5Ztable"]').find_element_by_css_selector('[id="PlanManagerView$Hirc_valueCell0"]')
        if WBS.text=="4016737":
            time.sleep(1)
            break
    except:
        time.sleep(3)
driver.find_element_by_css_selector('#navbar > div.nav-menu > ul > li:nth-child(5) > a').click()
time.sleep(2)
driver.find_element_by_css_selector('[src="http://w3.huawei.com/pm/resource/ui/lib/smartclient-Lib/skins/Enterprise/images/actions/filter.png"]').click()
time.sleep(1)
move_bar=driver.find_element_by_css_selector('[src="http://w3.huawei.com/pm/resource/ui/lib/smartclient-Lib/skins/Enterprise/images/Scrollbar/hthumb_grip.png"]')
ActionChains(driver).drag_and_drop_by_offset(move_bar, 921, 0).perform()
time.sleep(1)
driver.find_element_by_css_selector('#isc_CC').click()
driver.find_element_by_css_selector('[id="isc_PickListMenu_3_row_3"]').click()
driver.find_element_by_css_selector('#isc_DR').send_keys('Yifan')
time.sleep(1)
driver.find_element_by_css_selector('[src="http://w3.huawei.com/pm/resource/ui/lib/smartclient-Lib/skins/Enterprise/images/RecordEditor/filter.png"]').click()
ActionChains(driver).drag_and_drop_by_offset(move_bar, -921, 0).perform()
time.sleep(3)
driver.find_element_by_css_selector('[id="WBS$Activity$MassUpdate_wbsControlCheckBoxName"]').click()
time.sleep(1)
right_click_place=driver.find_element_by_css_selector('[id="WBS$Activity$MassUpdate_wbsCheckBoxName_0"]')
ActionChains(driver).context_click(right_click_place).perform()
driver.find_element_by_css_selector('[src="http://w3.huawei.com/pm/images/releaseWbs.png"]').click()
time.sleep(1)
driver.find_element_by_css_selector('[id="btnSubmit"]').click()


#table check box ID from 0
id="WBS$Activity$MassUpdate_wbsCheckBoxName_0"
#dialog box
class="windowBody"

Test_sample from website
username = driver.find_element_by_css_selector("input[tabindex='1']")  
password = driver.find_element_by_css_selector("input[tabindex='2']")